In [20]:
!pip install -q pandas
import pandas as pd
import re

You should consider upgrading via the '/Users/derekweitzel/tmp/gracc-compare/venv/bin/python3.9 -m pip install --upgrade pip' command.


In [55]:
# Read in the 2 csv files
gracc_df = pd.read_csv('GRACC-OSPool-July-2021-2022.csv', thousands=',')
gracc_df.rename(columns={'Sum of CoreHours': 'GRACC Hours'}, inplace=True)


ospool_df = pd.read_csv('HTCondor-OSPool-July-2021-2022.csv')
ospool_df.rename(columns={'Hours': 'OSPool Hours'}, inplace=True)
#ospool_df.head()


In [56]:
# Merge the 2 dataframes
missing_in_ospool = set(gracc_df['Project'].tolist()) - set(ospool_df['Project'].tolist())
missing_in_gracc = set(ospool_df['Project'].tolist()) - set(gracc_df['Project'].tolist())
#print("Missing in OSPool: " + str(missing_in_ospool))
#print("Missing in GRACC: " + str(missing_in_gracc))
gracc_ospool_df = pd.merge(gracc_df, ospool_df, on='Project')

# Now add in the missing projects
for project in missing_in_ospool:
    gracc_ospool_df = pd.concat([gracc_ospool_df, gracc_df[gracc_df['Project'] == project]], axis=0, join='outer', ignore_index=True)
    gracc_ospool_df.loc[gracc_ospool_df['Project'] == project, 'OSPool Hours'] = 0
for project in missing_in_gracc:
    gracc_ospool_df = pd.concat([gracc_ospool_df, ospool_df[ospool_df['Project'] == project]], axis=0, join='outer', ignore_index=True)
    gracc_ospool_df.loc[gracc_ospool_df['Project'] == project, 'GRACC Hours'] = 0

gracc_ospool_df


,Project,Field of Science,GRACC Hours,Rank,OSPool Hours
0,Hawaii_Doetinchem,Physics,19750257.6,1.0,19676436.0
1,TG-CHE200122,Chemistry,19157805.6,3.0,17166300.0
2,WSU_3DHydro,Physics,18158021.9,2.0,17773241.0
3,spt.all,Astronomy,11734939.9,39.0,900895.0
4,eht,Astronomy,9728498.4,5.0,9699502.0
...,...,...,...,...,...
233,osg.NeuroscienceGateway,NaN,0.0,49.0,511019.0
234,snowmass21.energy,NaN,0.0,81.0,31223.0
235,duke.wfirst,NaN,0.0,88.0,21142.0
236,UTEP_DeBlasio,NaN,0.0,38.0,903608.0


In [57]:
# Calculate the percent difference between gracc hours and ospool hours
gracc_ospool_df['Percent Difference'] = (gracc_ospool_df['GRACC Hours'] - gracc_ospool_df['OSPool Hours']) / gracc_ospool_df['OSPool Hours']

In [58]:
gracc_ospool_df.to_csv('gracc_ospool_df.csv')